# Step 2.3: Feature Extraction
## Windowed Feature Extraction from E4 Signals

In [1]:
import os
import pandas as pd
import numpy as np
from pathlib import Path
import pickle
from scipy import stats, signal
from scipy.fft import fft
import warnings
warnings.filterwarnings('ignore')

BASE_PATH = Path('/home/alvaro-ibarra/smartwatch-stress-detection')
WESAD_PATH = BASE_PATH / 'WESAD'
EPM_PATH = BASE_PATH / 'EPM-E4' / 'empatica_wearable_data' / 'raw'
PHYSIONET_PATH = BASE_PATH / 'wearable-device-dataset' / 'wearable-device-dataset-from-induced-stress-and-structured-exercise-sessions-1.0.1' / 'Wearable_Dataset'
OUTPUT_PATH = BASE_PATH / 'data' / 'processed' / 'windowed_features'

# Window parameters
WINDOW_SIZE_SEC = 60  # 60 seconds
OVERLAP = 0.5  # 50% overlap

# Sampling rates
SR_BVP = 64
SR_EDA = 4
SR_TEMP = 4
SR_ACC = 32
SR_HR = 1

In [15]:
def load_e4_signal(filepath):
    """Load E4 CSV signal file."""
    try:
        with open(filepath, 'r') as f:
            lines = f.readlines()
        timestamp = lines[0].strip().split(',')[0]
        sr = float(lines[1].strip().split(',')[0])
        data = pd.read_csv(filepath, skiprows=2, header=None)
        return data.values, sr, timestamp
    except Exception as e:
        print(f"Error loading {filepath}: {e}")
        return None, None, None

def get_windows(data, sr, window_sec, overlap):
    """Generate sliding windows."""
    window_samples = int(window_sec * sr)
    step = int(window_samples * (1 - overlap))
    windows = []
    for start in range(0, len(data) - window_samples + 1, step):
        windows.append(data[start:start + window_samples])
    return windows

In [3]:
# BVP/HR Feature Extraction
def extract_bvp_features(bvp_window, hr_window=None):
    """Extract BVP and HR features."""
    features = {}
    bvp = bvp_window.flatten()
    
    # BVP statistics
    features['bvp_mean'] = np.mean(bvp)
    features['bvp_std'] = np.std(bvp)
    features['bvp_min'] = np.min(bvp)
    features['bvp_max'] = np.max(bvp)
    features['bvp_range'] = features['bvp_max'] - features['bvp_min']
    
    # Peak detection for HRV
    peaks, _ = signal.find_peaks(bvp, distance=int(SR_BVP * 0.5))
    if len(peaks) > 1:
        ibi = np.diff(peaks) / SR_BVP * 1000  # IBI in ms
        features['hr_mean'] = 60000 / np.mean(ibi) if np.mean(ibi) > 0 else np.nan
        features['hr_std'] = np.std(60000 / ibi) if len(ibi) > 1 else 0
        features['hrv_rmssd'] = np.sqrt(np.mean(np.diff(ibi)**2)) if len(ibi) > 1 else 0
        features['hrv_sdnn'] = np.std(ibi) if len(ibi) > 1 else 0
        nn_diff = np.abs(np.diff(ibi))
        features['hrv_pnn50'] = np.sum(nn_diff > 50) / len(nn_diff) * 100 if len(nn_diff) > 0 else 0
    else:
        features['hr_mean'] = np.nan
        features['hr_std'] = 0
        features['hrv_rmssd'] = 0
        features['hrv_sdnn'] = 0
        features['hrv_pnn50'] = 0
    
    # Frequency domain (LF/HF ratio)
    try:
        freqs, psd = signal.welch(bvp, fs=SR_BVP, nperseg=min(256, len(bvp)))
        lf_mask = (freqs >= 0.04) & (freqs < 0.15)
        hf_mask = (freqs >= 0.15) & (freqs < 0.4)
        lf_power = np.trapezoid(psd[lf_mask], freqs[lf_mask]) if np.sum(lf_mask) > 1 else 0
        hf_power = np.trapezoid(psd[hf_mask], freqs[hf_mask]) if np.sum(hf_mask) > 1 else 0
        features['hrv_lf_hf_ratio'] = lf_power / hf_power if hf_power > 0 else np.nan
    except:
        features['hrv_lf_hf_ratio'] = np.nan
    
    return features

In [4]:
# EDA Feature Extraction
def extract_eda_features(eda_window):
    """Extract EDA features."""
    features = {}
    eda = eda_window.flatten()
    
    # Basic statistics
    features['eda_mean'] = np.mean(eda)
    features['eda_std'] = np.std(eda)
    features['eda_min'] = np.min(eda)
    features['eda_max'] = np.max(eda)
    features['eda_range'] = features['eda_max'] - features['eda_min']
    
    # SCR detection (simple peak detection)
    eda_diff = np.diff(eda)
    threshold = np.std(eda_diff) * 0.5
    peaks, properties = signal.find_peaks(eda, prominence=threshold, distance=int(SR_EDA * 1))
    
    features['eda_scr_count'] = len(peaks)
    if len(peaks) > 0 and 'prominences' in properties:
        features['eda_scr_amp_mean'] = np.mean(properties['prominences'])
    else:
        features['eda_scr_amp_mean'] = 0
    
    # Tonic/Phasic approximation (simple low-pass for tonic)
    try:
        b, a = signal.butter(2, 0.05, btype='low', fs=SR_EDA)
        tonic = signal.filtfilt(b, a, eda)
        phasic = eda - tonic
        features['eda_tonic_mean'] = np.mean(tonic)
        features['eda_phasic_mean'] = np.mean(np.abs(phasic))
    except:
        features['eda_tonic_mean'] = features['eda_mean']
        features['eda_phasic_mean'] = 0
    
    # Slope
    features['eda_slope'] = (eda[-1] - eda[0]) / len(eda) * SR_EDA
    
    return features

In [5]:
# Temperature Feature Extraction
def extract_temp_features(temp_window):
    """Extract temperature features."""
    features = {}
    temp = temp_window.flatten()
    
    features['temp_mean'] = np.mean(temp)
    features['temp_std'] = np.std(temp)
    features['temp_min'] = np.min(temp)
    features['temp_max'] = np.max(temp)
    features['temp_range'] = features['temp_max'] - features['temp_min']
    features['temp_slope'] = (temp[-1] - temp[0]) / len(temp) * SR_TEMP
    
    return features

In [6]:
# Accelerometer Feature Extraction
def extract_acc_features(acc_window):
    """Extract accelerometer features."""
    features = {}
    
    if acc_window.shape[1] >= 3:
        x, y, z = acc_window[:, 0], acc_window[:, 1], acc_window[:, 2]
    else:
        x = y = z = acc_window.flatten()
    
    # Magnitude
    magnitude = np.sqrt(x**2 + y**2 + z**2)
    
    features['acc_mag_mean'] = np.mean(magnitude)
    features['acc_mag_std'] = np.std(magnitude)
    features['acc_mag_min'] = np.min(magnitude)
    features['acc_mag_max'] = np.max(magnitude)
    
    # Per-axis
    features['acc_x_mean'] = np.mean(x)
    features['acc_y_mean'] = np.mean(y)
    features['acc_z_mean'] = np.mean(z)
    features['acc_x_std'] = np.std(x)
    features['acc_y_std'] = np.std(y)
    features['acc_z_std'] = np.std(z)
    
    # Signal Magnitude Area
    features['acc_sma'] = np.mean(np.abs(x) + np.abs(y) + np.abs(z))
    
    # Energy
    features['acc_energy'] = np.sum(magnitude**2) / len(magnitude)
    
    # Entropy
    hist, _ = np.histogram(magnitude, bins=10, density=True)
    hist = hist[hist > 0]
    features['acc_entropy'] = -np.sum(hist * np.log2(hist)) if len(hist) > 0 else 0
    
    return features

In [7]:
def extract_all_features(bvp, eda, temp, acc, window_idx):
    """Extract all features for a window."""
    features = {'window_id': window_idx}
    features.update(extract_bvp_features(bvp))
    features.update(extract_eda_features(eda))
    features.update(extract_temp_features(temp))
    features.update(extract_acc_features(acc))
    return features

## WESAD Feature Extraction

In [8]:
# WESAD label mapping
WESAD_LABELS = {1: 'Baseline', 2: 'Stress', 3: 'Amusement'}

wesad_subjects = sorted([d for d in os.listdir(WESAD_PATH) if d.startswith('S') and os.path.isdir(WESAD_PATH / d)], key=lambda x: int(x[1:]))
print(f"Processing {len(wesad_subjects)} WESAD subjects...")

Processing 15 WESAD subjects...


In [9]:
wesad_features = []

for subj in wesad_subjects:
    print(f"  Processing {subj}...", end=' ')
    
    # Load signals
    e4_path = WESAD_PATH / subj / f'{subj}_E4_Data'
    bvp_data, _, _ = load_e4_signal(e4_path / 'BVP.csv')
    eda_data, _, _ = load_e4_signal(e4_path / 'EDA.csv')
    temp_data, _, _ = load_e4_signal(e4_path / 'TEMP.csv')
    acc_data, _, _ = load_e4_signal(e4_path / 'ACC.csv')
    
    # Load labels from pkl
    with open(WESAD_PATH / subj / f'{subj}.pkl', 'rb') as f:
        pkl_data = pickle.load(f, encoding='latin1')
    labels = pkl_data['label']
    
    # Resample labels to match E4 BVP rate (labels are at 700Hz chest)
    label_ratio = len(labels) / len(bvp_data)
    
    # Create windows
    n_windows = int((len(bvp_data) - WINDOW_SIZE_SEC * SR_BVP) / (WINDOW_SIZE_SEC * SR_BVP * (1 - OVERLAP))) + 1
    
    window_count = 0
    for w in range(n_windows):
        # Calculate indices for each signal
        bvp_start = int(w * WINDOW_SIZE_SEC * SR_BVP * (1 - OVERLAP))
        bvp_end = bvp_start + int(WINDOW_SIZE_SEC * SR_BVP)
        
        eda_start = int(w * WINDOW_SIZE_SEC * SR_EDA * (1 - OVERLAP))
        eda_end = eda_start + int(WINDOW_SIZE_SEC * SR_EDA)
        
        temp_start = int(w * WINDOW_SIZE_SEC * SR_TEMP * (1 - OVERLAP))
        temp_end = temp_start + int(WINDOW_SIZE_SEC * SR_TEMP)
        
        acc_start = int(w * WINDOW_SIZE_SEC * SR_ACC * (1 - OVERLAP))
        acc_end = acc_start + int(WINDOW_SIZE_SEC * SR_ACC)
        
        # Check bounds
        if bvp_end > len(bvp_data) or eda_end > len(eda_data) or temp_end > len(temp_data) or acc_end > len(acc_data):
            break
        
        # Get window label (majority vote)
        label_start = int(bvp_start * label_ratio)
        label_end = int(bvp_end * label_ratio)
        window_labels = labels[label_start:label_end]
        
        # Filter for valid labels (1, 2, 3)
        valid_labels = window_labels[(window_labels >= 1) & (window_labels <= 3)]
        if len(valid_labels) < len(window_labels) * 0.8:
            continue  # Skip windows with too many invalid labels
        
        majority_label = int(stats.mode(valid_labels, keepdims=False)[0])
        
        # Extract features
        features = extract_all_features(
            bvp_data[bvp_start:bvp_end],
            eda_data[eda_start:eda_end],
            temp_data[temp_start:temp_end],
            acc_data[acc_start:acc_end],
            window_count
        )
        
        features['subject_id'] = subj
        features['dataset'] = 'WESAD'
        features['label'] = WESAD_LABELS[majority_label]
        features['timestamp_start'] = bvp_start / SR_BVP
        features['timestamp_end'] = bvp_end / SR_BVP
        
        wesad_features.append(features)
        window_count += 1
    
    print(f"{window_count} windows")

wesad_features_df = pd.DataFrame(wesad_features)
print(f"\nTotal WESAD windows: {len(wesad_features_df)}")
print(wesad_features_df['label'].value_counts())

  Processing S2... 89 windows
  Processing S3... 81 windows
  Processing S4... 86 windows
  Processing S5... 86 windows
  Processing S6... 82 windows
  Processing S7... 87 windows
  Processing S8... 86 windows
  Processing S9... 84 windows
  Processing S10... 91 windows
  Processing S11... 87 windows
  Processing S13... 87 windows
  Processing S14... 90 windows
  Processing S15... 90 windows
  Processing S16... 90 windows
  Processing S17... 89 windows

Total WESAD windows: 1305
label
Baseline     707
Stress       389
Amusement    209
Name: count, dtype: int64


## EPM-E4 Feature Extraction

In [10]:
# EPM-E4: All data is emotion-elicited (no explicit baseline in E4 files)
# Labels come from folder structure and key_moments
epm_subjects = sorted([d for d in os.listdir(EPM_PATH) if os.path.isdir(EPM_PATH / d) and d != '.DS_Store'], key=lambda x: int(x))
print(f"Processing {len(epm_subjects)} EPM-E4 subjects...")

Processing 47 EPM-E4 subjects...


In [11]:
# Load key moments for reference
key_moments_path = BASE_PATH / 'EPM-E4' / 'key_moments'
key_moments = {}
for emotion in ['ANGER', 'FEAR', 'HAPPINESS', 'SADNESS']:
    km_df = pd.read_csv(key_moments_path / f'{emotion}.csv')
    key_moments[emotion] = km_df
    print(f"{emotion}: {len(km_df)} key moments")

ANGER: 5 key moments
FEAR: 9 key moments
HAPPINESS: 5 key moments
SADNESS: 5 key moments


In [12]:
epm_features = []

for subj in epm_subjects:
    print(f"  Processing {subj}...", end=' ')
    
    empatica_path = EPM_PATH / subj / 'empatica'
    
    # Load signals
    bvp_data, _, bvp_ts = load_e4_signal(empatica_path / 'BVP.csv')
    eda_data, _, _ = load_e4_signal(empatica_path / 'EDA.csv')
    temp_data, _, _ = load_e4_signal(empatica_path / 'TEMP.csv')
    acc_data, _, _ = load_e4_signal(empatica_path / 'ACC.csv')
    
    if bvp_data is None:
        print("skipped (no data)")
        continue
    
    # For EPM-E4, label all data as mixed emotional state
    # In practice, the entire recording is emotion elicitation
    # We'll assign based on the primary emotion being elicited
    
    # Create windows
    n_windows = int((len(bvp_data) - WINDOW_SIZE_SEC * SR_BVP) / (WINDOW_SIZE_SEC * SR_BVP * (1 - OVERLAP))) + 1
    
    window_count = 0
    for w in range(n_windows):
        bvp_start = int(w * WINDOW_SIZE_SEC * SR_BVP * (1 - OVERLAP))
        bvp_end = bvp_start + int(WINDOW_SIZE_SEC * SR_BVP)
        
        eda_start = int(w * WINDOW_SIZE_SEC * SR_EDA * (1 - OVERLAP))
        eda_end = eda_start + int(WINDOW_SIZE_SEC * SR_EDA)
        
        temp_start = int(w * WINDOW_SIZE_SEC * SR_TEMP * (1 - OVERLAP))
        temp_end = temp_start + int(WINDOW_SIZE_SEC * SR_TEMP)
        
        acc_start = int(w * WINDOW_SIZE_SEC * SR_ACC * (1 - OVERLAP))
        acc_end = acc_start + int(WINDOW_SIZE_SEC * SR_ACC)
        
        if bvp_end > len(bvp_data) or eda_end > len(eda_data) or temp_end > len(temp_data) or acc_end > len(acc_data):
            break
        
        features = extract_all_features(
            bvp_data[bvp_start:bvp_end],
            eda_data[eda_start:eda_end],
            temp_data[temp_start:temp_end],
            acc_data[acc_start:acc_end],
            window_count
        )
        
        features['subject_id'] = subj
        features['dataset'] = 'EPM-E4'
        features['label'] = 'Emotion'  # Will be refined with key_moments
        features['timestamp_start'] = bvp_start / SR_BVP
        features['timestamp_end'] = bvp_end / SR_BVP
        
        epm_features.append(features)
        window_count += 1
    
    print(f"{window_count} windows")

epm_features_df = pd.DataFrame(epm_features)
print(f"\nTotal EPM-E4 windows: {len(epm_features_df)}")

  Processing 1... 49 windows
  Processing 2... 46 windows
  Processing 4... 52 windows
  Processing 12... 60 windows
  Processing 14... 47 windows
  Processing 15... 49 windows
  Processing 16... 47 windows
  Processing 24... 54 windows
  Processing 25... 62 windows
  Processing 34... 52 windows
  Processing 35... 66 windows
  Processing 36... 47 windows
  Processing 39... 62 windows
  Processing 40... 49 windows
  Processing 45... 51 windows
  Processing 47... 55 windows
  Processing 49... 49 windows
  Processing 55... 50 windows
  Processing 56... 50 windows
  Processing 61... 49 windows
  Processing 66... 49 windows
  Processing 68... 51 windows
  Processing 72... 68 windows
  Processing 76... 55 windows
  Processing 77... 53 windows
  Processing 78... 70 windows
  Processing 79... 53 windows
  Processing 81... 45 windows
  Processing 86... 50 windows
  Processing 91... 52 windows
  Processing 92... 52 windows
  Processing 103... 50 windows
  Processing 106... 72 windows
  Processin

## PhysioNet Feature Extraction

In [13]:
# PhysioNet: Label from protocol folder
PHYSIONET_LABELS = {'STRESS': 'Stress', 'AEROBIC': 'Aerobic', 'ANAEROBIC': 'Anaerobic'}

# Load exclusion list
excluded_df = pd.read_csv(BASE_PATH / 'outputs' / 'tables' / 'excluded_subjects.csv')
excluded_stress = excluded_df[(excluded_df['action'] == 'EXCLUDE') & (excluded_df['protocol'] == 'STRESS')]['subject_id'].tolist()
excluded_aerobic = excluded_df[(excluded_df['action'] == 'EXCLUDE') & (excluded_df['protocol'] == 'AEROBIC')]['subject_id'].tolist()

print(f"Excluded from STRESS: {excluded_stress}")
print(f"Excluded from AEROBIC: {excluded_aerobic}")

Excluded from STRESS: ['f07']
Excluded from AEROBIC: ['S12']


In [16]:
physionet_features = []

for protocol in ['STRESS', 'AEROBIC', 'ANAEROBIC']:
    protocol_path = PHYSIONET_PATH / protocol
    subjects = sorted([d for d in os.listdir(protocol_path) if os.path.isdir(protocol_path / d)])
    
    print(f"\nProcessing {protocol} ({len(subjects)} subjects)...")
    
    for subj in subjects:
        # Check exclusion
        base_subj = subj.replace('_a', '').replace('_b', '')
        if protocol == 'STRESS' and base_subj in excluded_stress:
            continue
        if protocol == 'AEROBIC' and base_subj in excluded_aerobic:
            continue
        
        print(f"  Processing {subj}...", end=' ')
        
        subj_path = protocol_path / subj
        
        bvp_data, _, _ = load_e4_signal(subj_path / 'BVP.csv')
        eda_data, _, _ = load_e4_signal(subj_path / 'EDA.csv')
        temp_data, _, _ = load_e4_signal(subj_path / 'TEMP.csv')
        acc_data, _, _ = load_e4_signal(subj_path / 'ACC.csv')
        
        if bvp_data is None:
            print("skipped (no data)")
            continue
        
        n_windows = int((len(bvp_data) - WINDOW_SIZE_SEC * SR_BVP) / (WINDOW_SIZE_SEC * SR_BVP * (1 - OVERLAP))) + 1
        
        window_count = 0
        for w in range(n_windows):
            bvp_start = int(w * WINDOW_SIZE_SEC * SR_BVP * (1 - OVERLAP))
            bvp_end = bvp_start + int(WINDOW_SIZE_SEC * SR_BVP)
            
            eda_start = int(w * WINDOW_SIZE_SEC * SR_EDA * (1 - OVERLAP))
            eda_end = eda_start + int(WINDOW_SIZE_SEC * SR_EDA)
            
            temp_start = int(w * WINDOW_SIZE_SEC * SR_TEMP * (1 - OVERLAP))
            temp_end = temp_start + int(WINDOW_SIZE_SEC * SR_TEMP)
            
            acc_start = int(w * WINDOW_SIZE_SEC * SR_ACC * (1 - OVERLAP))
            acc_end = acc_start + int(WINDOW_SIZE_SEC * SR_ACC)
            
            if bvp_end > len(bvp_data) or eda_end > len(eda_data) or temp_end > len(temp_data) or acc_end > len(acc_data):
                break
            
            features = extract_all_features(
                bvp_data[bvp_start:bvp_end],
                eda_data[eda_start:eda_end],
                temp_data[temp_start:temp_end],
                acc_data[acc_start:acc_end],
                window_count
            )
            
            features['subject_id'] = subj
            features['dataset'] = 'PhysioNet'
            features['label'] = PHYSIONET_LABELS[protocol]
            features['timestamp_start'] = bvp_start / SR_BVP
            features['timestamp_end'] = bvp_end / SR_BVP
            
            physionet_features.append(features)
            window_count += 1
        
        print(f"{window_count} windows")

physionet_features_df = pd.DataFrame(physionet_features)
print(f"\nTotal PhysioNet windows: {len(physionet_features_df)}")
print(physionet_features_df['label'].value_counts())


Processing STRESS (37 subjects)...
  Processing S01... 73 windows
  Processing S02... 102 windows
  Processing S03... 51 windows
  Processing S04... 56 windows
  Processing S05... 51 windows
  Processing S06... 47 windows
  Processing S07... 46 windows
  Processing S08... 46 windows
  Processing S09... 47 windows
  Processing S10... 54 windows
  Processing S11... 45 windows
  Processing S12... 52 windows
  Processing S13... 53 windows
  Processing S14... 55 windows
  Processing S15... 53 windows
  Processing S16... 54 windows
  Processing S17... 52 windows
  Processing S18... 53 windows
  Processing f01... 107 windows
  Processing f02... 98 windows
  Processing f03... 118 windows
  Processing f04... 120 windows
  Processing f05... 152 windows
  Processing f06... 104 windows
  Processing f08... 105 windows
  Processing f09... 106 windows
  Processing f10... 90 windows
  Processing f11... 103 windows
  Processing f12... 159 windows
  Processing f13... 127 windows
  Processing f14_a... 2

## Save Windowed Features

In [17]:
# Save individual dataset features
wesad_features_df.to_csv(OUTPUT_PATH / 'WESAD_windowed_features.csv', index=False)
epm_features_df.to_csv(OUTPUT_PATH / 'EPM4_windowed_features.csv', index=False)
physionet_features_df.to_csv(OUTPUT_PATH / 'PhysioNet_windowed_features.csv', index=False)

print("Saved:")
print(f"  WESAD_windowed_features.csv: {len(wesad_features_df)} rows")
print(f"  EPM4_windowed_features.csv: {len(epm_features_df)} rows")
print(f"  PhysioNet_windowed_features.csv: {len(physionet_features_df)} rows")

Saved:
  WESAD_windowed_features.csv: 1305 rows
  EPM4_windowed_features.csv: 2510 rows
  PhysioNet_windowed_features.csv: 6696 rows


In [18]:
# Summary
print("="*60)
print("STEP 2.3 COMPLETE: Feature Extraction")
print("="*60)
print(f"\nWindow size: {WINDOW_SIZE_SEC}s, Overlap: {int(OVERLAP*100)}%")
print(f"\nFeatures extracted: {len([c for c in wesad_features_df.columns if c not in ['subject_id', 'dataset', 'label', 'window_id', 'timestamp_start', 'timestamp_end']])}")
print(f"\nTotal windows:")
print(f"  WESAD: {len(wesad_features_df)}")
print(f"  EPM-E4: {len(epm_features_df)}")
print(f"  PhysioNet: {len(physionet_features_df)}")
print(f"  TOTAL: {len(wesad_features_df) + len(epm_features_df) + len(physionet_features_df)}")
print("="*60)

STEP 2.3 COMPLETE: Feature Extraction

Window size: 60s, Overlap: 50%

Features extracted: 40

Total windows:
  WESAD: 1305
  EPM-E4: 2510
  PhysioNet: 6696
  TOTAL: 10511
